<a href="https://colab.research.google.com/github/andyarnell/Global_ecological_zones_mapping/blob/main/gez_comparison_sankey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Aim: create Sankey plots to illustrate magnitude of climate zone changes to help compare GEZ update options

In [ ]:
# Change the current working directory to "/content".
%cd "/content"

# Clone the GitHub repository "sepal_mgci" into the current directory.
# NB 'fatal' error on reruns are typically just saying it already exists
!git clone https://github.com/andyarnell/sankee.git

/content
fatal: destination path 'sankee' already exists and is not an empty directory.


In [38]:
# to automatically reload modules.
%load_ext autoreload

# Set to reload all modules before executing code.
%autoreload 2


# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

# install_if_not_exists("sankee")
install_if_not_exists("os")

import os

%cd ..

print(os.listdir())

import sys

%cd "/content/sankee"

import sankee

import ee # google earth engine

import geemap

from google.colab import output

output.enable_custom_widget_manager()

gee_project_name = "ee-andyarnellgee"

ee.Authenticate()

ee.Initialize(project=gee_project_name)

# # # Generate the Sankey diagram from the two images
# sankee.sankify(title=title,
#     image_list=image_list,
#     region=aoi.geometry().bounds(),
#     band=band,
#     labels=labels,
#     palette=palette,
#     scale=scale,
#     n=2000,
#     seed=0
# )


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
os is already installed.
/content
['.config', 'sankee', 'sample_data']
/content/sankee


In [39]:
chelsa_climate_1981_2010 = ee.ImageCollection("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/chelsa_tas_1981_2010");
chelsa_pet = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/chelsa_pet_penman_mean_1981-2010_V_2_1");
worldclim_climate = ee.ImageCollection("WORLDCLIM/V1/MONTHLY");
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE");
mountain_belts = ee.Image("users/xavidelamo/SDG1542_Mntn_BioclimaticBelts");#from Korner et al
hzl_contemp = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/life_zones_contemporary");
# admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0");
# table = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");

In [50]:
import ee

# Initialize the Earth Engine module
ee.Initialize()

# Load the existing GEZ data showing level 2
gez_2010_level2 = ee.Image('users/bornToBeAlive/gez_2010_wgs84').selfMask() # includes removing lakes etc (val 90)
gez_2010_level2 = gez_2010_level2.updateMask(gez_2010_level2.neq(90))

# Load the existing GEZ data showing level 1 (approximate climate zones)
gez_2010_level1 = gez_2010_level2.divide(10).floor().int().selfMask()

# Load the Holdridge Life Zones (adapted from Elsen et al 2022)
hzl_contemp = ee.Image("projects/ee-andyarnellgee/assets/p0001_global_ecological_zones_update/raw/life_zones_contemporary")

# Create a simplified version of the Holdridge Life Zones to approximate GEZ level 1
hzl_contemp_simple = hzl_contemp.divide(10).floor().int().selfMask().updateMask(gez_2010_level1)

hzl_contemp_simple_remap = hzl_contemp_simple.remap([1,2,3,4,5,6],[6,5,4,3,2,1],0) #remap values to match gez etc



# Function to calculate number of months above a temperature threshold
def calc_number_of_months_above(monthly_image_col, band_name_tavg, temp_threshold):
    def map_threshold(image):
        image_tavg = image.select(band_name_tavg)
        image_gte_thresh = image_tavg.gte(temp_threshold)
        return image_gte_thresh

    monthly_image_col_tavg = monthly_image_col.select(band_name_tavg)
    monthly_image_col_tavg_gte_thresh = monthly_image_col_tavg.map(map_threshold)
    count_of_months_above = monthly_image_col_tavg_gte_thresh.sum()
    return count_of_months_above

# Function to classify Köppen-Trewartha climate zones
def classify_koppen_trewartha(monthly_image_col, band_name_tavg, prec_avg, pet_avg, high_thresh, low_thresh):
    # Calculate number of months above 18 degrees and 10 degrees
    number_of_months_above_18_degrees = calc_number_of_months_above(monthly_image_col, band_name_tavg, high_thresh)
    number_of_months_above_10_degrees = calc_number_of_months_above(monthly_image_col, band_name_tavg, low_thresh)

    # Define Köppen-Trewartha rules
    threshold_18_deg = 12  # standard kt val :12 (but for toggling to get similar to GEZ level 1, 8 is closer!)
    high_threshold_10_deg = 8  # standard kt val :8
    middle_threshold_10_deg = 4  # standard kt val :4
    low_threshold_10_deg = 1  # standard kt val : 1

    # Define Köppen-Trewartha rules
    tropical = number_of_months_above_18_degrees.gte(threshold_18_deg)
    subtropical = number_of_months_above_10_degrees.gte(high_threshold_10_deg).And(number_of_months_above_18_degrees.lt(threshold_18_deg))
    temperate = number_of_months_above_10_degrees.gte(middle_threshold_10_deg).And(number_of_months_above_10_degrees.lt(high_threshold_10_deg))
    boreal = number_of_months_above_10_degrees.gte(low_threshold_10_deg).And(number_of_months_above_10_degrees.lt(middle_threshold_10_deg))
    polar = number_of_months_above_10_degrees.lt(low_threshold_10_deg)

    # Combine rules to create zones
    koppenTrewarthaZones = ee.Image(0) \
        .where(tropical, 1) \
        .where(subtropical, 2) \
        .where(temperate, 3) \
        .where(boreal, 4) \
        .where(polar, 5) \
        .selfMask()

    # Return the image with Köppen-Trewartha zones
    return koppenTrewarthaZones.rename('koppen_trewartha')



# Data prep

# Potential evapotranspiration
terraclimate_pet_avg = terraclimate.select('pet').filter(ee.Filter.date('1960-01-01', '1991-01-01')).mean()

# Precipitation
worldclim_climate_prec_avg = worldclim_climate.select('prec').mean().updateMask(gez_2010_level1)



# Analysis
# Apply classify_koppen_trewartha function to worldclim
worldclim_kt = classify_koppen_trewartha(worldclim_climate,
                                                      'tavg',
                                                      worldclim_climate_prec_avg,
                                                      terraclimate_pet_avg,  # as not included in worldclim
                                                      180,
                                                      100)

# Apply classify_koppen_trewartha function to chelsa
chelsa_kt = classify_koppen_trewartha(chelsa_climate_1981_2010,
                                                      'b1',
                                                      worldclim_climate_prec_avg,  # using for temp interim
                                                      chelsa_pet,
                                                      18,
                                                      10)

# Mask to terrestrial areas (based on worldclim extent for now)
chelsa_kt = chelsa_kt.updateMask(worldclim_kt.gt(0)).updateMask(gez_2010_level1)

gez_mountains = (
    gez_2010_level2.eq(16)
    .Or(gez_2010_level2.eq(25))
    .Or(gez_2010_level2.eq(35))
    .Or(gez_2010_level2.eq(43))
)

gez_2010_level1 = gez_2010_level1.updateMask(chelsa_kt) # all layers should have same extent now

In [51]:


# Create a map
Map = geemap.Map()

# Add layers to the map
Map.addLayer(gez_2010_level1.randomVisualizer(), {}, "gez_2010_level1")
Map.addLayer(gez_2010_level2.randomVisualizer(), {}, "gez_2010_level2")

# Add data-driven zones based on climate datasets
Map.addLayer(worldclim_kt.select('koppen_trewartha').randomVisualizer(),{},'Worldclim Köppen-Trewartha Climate Zones');

Map.addLayer(chelsa_kt.select('koppen_trewartha').randomVisualizer(),{},'Chelsa Köppen-Trewartha Climate Zones');

# Map.addLayer(hzl_contemp, {}, "life zones contemporary - all")
Map.addLayer(hzl_contemp_simple_remap.randomVisualizer(), {}, "holdridge life zones contemporary - simple")


# # Load two images
# aoi = ee.Geometry.Point([-122.30239918572622, 44.802882471354316]).buffer(5000000)

# aoi = ee.Geometry.Polygon([xMin, yMin, xMax, yMax], None,False)
world = ee.Geometry.Polygon([
    [-180, 90], [0, 90], [180, 90],
    [180,-90], [0, -90], [-180,-90]],
    None, False);

# aoi = world
# Map.addLayer(aoi,{'color': 'FF0000'},"aoi")


gez_mountains_inverse_mask = gez_mountains.eq(0)

# Map.addLayer(gez_mountains_inverse_mask, {}, "gez_mountains_inverse_mask ")

mountain_belts_inverse_mask = mountain_belts.unmask(0).eq(0)

# Map.addLayer(mountain_belts_inverse_mask, {}, "mountain_belts_inverse_mask ")

mountain_mask = ee.ImageCollection(# get mask of mountains in both layers
    [mountain_belts_inverse_mask,
    gez_mountains_inverse_mask]).min()

# Map.addLayer(mountain_mask, {}, "mountain_mask ")


#versions without mountains
gez_2010_level1_no_mountains = gez_2010_level1.select(0).rename("b1").updateMask(mountain_mask)

worldclim_kt_no_mountains = worldclim_kt.select(0).rename("b1").updateMask(mountain_mask)

chelsa_kt_no_mountains =chelsa_kt.select(0).rename("b1").updateMask(mountain_mask)

hzl_contemp_simple_remap_no_mountains = hzl_contemp_simple_remap.rename("b1").updateMask(mountain_mask)


# Add layers to the map
Map.addLayer(gez_2010_level1_no_mountains.randomVisualizer(), {}, "gez_2010_level1")

# Add data-driven zones based on climate datasets
Map.addLayer(worldclim_kt_no_mountains.randomVisualizer(),{},'Worldclim Köppen-Trewartha Climate Zones');

Map.addLayer(chelsa_kt_no_mountains.randomVisualizer(),{},'Chelsa Köppen-Trewartha Climate Zones');

# Map.addLayer(hzl_contemp, {}, "life zones contemporary - all")
Map.addLayer(hzl_contemp_simple_remap_no_mountains.randomVisualizer(), {}, "holdridge life zones contemporary - simple")



# Display the map
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [42]:
# Define the band name and the class labels and colors corresponding to each pixel value.
band = "b1"

labels = {
    1: "tropical",
    2: "subtropical",
    3: "temperate",
    4: "boreal",
    5: "polar",
}
palette = {
    # 0: "#419BDF",
    1: "#397D49",
    2: "#88B053",
    3: "#7A87C6",
    4: "#E49635",
    5: "#DFC35A",
    # 6: "#C4281B",
    # 7: "#A59B8F",
    # 8: "#B39FE1"
}


In [43]:
factor =10 # to coarsen the resolution of the images to be sampled. 1 is native resolution

scale = gez.select(0).projection().nominalScale().multiply(ee.Number(factor)).getInfo()

print ("scale: ", scale)


admin_bounds = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");

# Create a dictionary mapping index to region name
region_dict = {
    0: "Africa",
    1: "E Asia",
    2: "Europe",
    3: "N Asia",
    4: "S Asia",
    5: "Oceania",
    6: "SE Asia",
    7: "SW Asia",
    8: "Australia",
    9: "Caribbean",
    10: "Antarctica",
    11: "S Atlantic",
    12: "Central Asia",
    13: "Indian Ocean",
    14: "North America",
    15: "South America",
    16: "Central America"
}

# Access the value for a specific key

index = 3  # Example key

title="Transitions GEZ Level 1 to Chelsa KT, to WorldClim KT  (from left to right)"

image_list = [gez, chelsa_kt, worldclim_kt]

region_name = region_dict[index]

print(f"Region name: {region_name} (for index {index})")


aoi = admin_bounds.filter(ee.Filter.eq("wld_rgn",region_name))


print ("number of countries: ",aoi.size().getInfo())

point_asset ="users/frarssuser1/WorldESTIMATES6Dec21_V1_GEEv3_/WorldESTIMATES6Dec21_V1_GEEv5_" # RSS points

# Generate the Sankey diagram from the two images
sankee.sankify(title=title,
    image_list=image_list,
    region=aoi.geometry().bounds(),
    band=band,
    labels=labels,
    palette=palette,
    scale=scale,
    # n=2000,
    seed=1,
    point_asset=point_asset,
    proportion = .1
)


scale:  1113.1949079327358
Region name: N Asia (for index 3)
number of countries:  5


SamplingError: The band `b1` was not found in all images. Choose from [].